In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import warnings


# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Import Python Module
from CRUD import AnimalShelter



###########################
# Data Manipulation / Model
###########################

# my Python module already has the args for the server
shelter = AnimalShelter()

warnings.simplefilter("ignore", FutureWarning)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(
    shelter.read({
        # Filter out dogs that have died or not available.
        "outcome_type": {
            "$nin": ["Euthanasia", "Died"]
        }
    })
)

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

# Reg Expression to extract dog ages
age_split = df["age_upon_outcome"].str.strip().str.split(" ", expand=True)
age_num = age_split[0].astype(float)

# Extract the number
age_unit = (
    age_split[1]
    .str.lower()
    .str.rstrip("s")
)

# Convert the units into a year format
unit_to_year = {
    "year": 1.0,
    "month": 1.0/12,
    "week": 1.0/52,
    "day": 1.0/365
}

# compute a single numeric column
df["age_years"] = age_num * age_unit.map(unit_to_year)

# Precompute full counts for “All Animals”
counts_all = (
    df
    .groupby(["animal_type","breed"])
    .size()
    .reset_index(name="count")
)

initial_fig = px.sunburst(
    counts_all,
    path=["animal_type", "breed"],
    values="count",
    branchvalues="total",
    title="Animals by Type & Breed"
)
initial_fig.update_layout(margin=dict(t=40, l=0, r=0, b=0))

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)
# print(df[["age_upon_outcome", "age_years"]].head())




#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

# Specific profiles for dog training
RESCUE_OPTIONS = [
    {"label": "All Animals", "value": "ALL"},
    {"label": "Water Rescue", "value": "WATER"},
    {"label": "Mountain/Wilderness", "value": "MOUNTAIN"},
    {"label": "Disaster Recovery", "value": "DISASTER"},
    {"label": "Scent Tracking", "value": "SCENT"},
]

# I used ChatGPT to find out the 'best' or most used dogs for each catagory.
BREED_MAP = {
    "WATER": ["Labrador Retriever", "Golden Retriever", "Newfoundland"],
    "MOUNTAIN": ["German Shepherd", "Labrador Retriever"],
    "DISASTER": ["German Shepherd", "Belgian Malinois", "Labrador Retriever", "Bloodhounds"],
    "SCENT": ["Bloodhound", "Beagle", "Basset Hound", "Coonhound"],
}

banner = html.Div(
    html.H1(
        "Grazioso Salvare",
        style={"fontWeight": "bold"}
    ),
    style={
        "backgroundColor": "teal", 
        "color": "white",        
        "textAlign": "center", 
        "padding": "20px",   
        "borderRadius": "8px"    
    }
)

app.layout = html.Div([
    
    html.Div(id='hidden-div', style={'display': 'none'}),
    
    banner,
    
    html.Marquee("Submitted by Ronny Z. Valtonen"),
    
    html.H2(
        "Welcome to the Animal Shelter Dashboard!",
        style={
            'color': 'teal',
            'fontFamily': 'Comic Sans MS',
            'textAlign': 'center'
        }
    ),
    
    html.Img(
        src="/assets/Grazioso_Salvare_Logo.png",
        style={'width': '150px', 'display': 'block', 'margin': 'auto'}
    ),
    
    html.Div(
        [
            dcc.Dropdown(
                id="dog-rescue-dropdown",
                className="dark-dropdown",
                options=RESCUE_OPTIONS,
                value="ALL",
                searchable=False,
                placeholder="Select Search & Rescue"
            ),
            dcc.Checklist(
                id="include-mixes",
                options=[{"label": "Include Mixes", "value": "MIXES"}],
                value=[],
                inputStyle={"margin-right": "4px"},
                labelStyle={"display": "inline-block", "margin-left": "20px"}
            )
        ],
        style={"textAlign": "left", "margin": "20px 0"}
    ),
    
    html.Div(
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        sort_by=[{"column_id": "rec_num", "direction": "asc"}],
        page_size=10,
        filter_action='native',
        sort_action='native',
        row_selectable='single',
        selected_rows=[0],
        column_selectable='multi',
        style_header={
            "backgroundColor": "#444444",
            "color": "white",
            "fontWeight": "bold"
        },
        style_cell={
            "backgroundColor": "#333333",
            "color": "white",
            "border": "none",
            "textAlign": "left",
            "padding": "5px"
        },
        style_data_conditional=[
            {
                "if": {"row_index": "odd"},
                "backgroundColor": "#2a2a2a"
            }
        ],
        style_table={"overflowX": "auto", "boxShadow": "0 0 10px rgba(0, 123, 255, 0.6)", "borderRadius": "8px"}
    ),
    style={"marginBottom": "30px"}
    ),
        html.Div([
        # Sunburst chart on the left
        html.Div(
            dcc.Graph(id="sunburst-chart", figure=initial_fig),
            style={"flex": 1, "minWidth": 0}
        ),

        # Geo-map on the right
        html.Div(
            id='map-id',
            style={"flex": 1, "minWidth": 0, "height": "500px"}
        )
    ], style={
        "display": "flex",
        "alignItems": "flex-start",
        "gap": "20px"
    })
],
    style={
      "color": "white",
      "minHeight": "100vh",
      "padding": "20px"
    }

)

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will filter specific dog profiles
@app.callback(
    [
        Output("datatable-id", "data"),
        Output("sunburst-chart", "figure"),
    ],
    [
        Input("dog-rescue-dropdown", "value"),
        Input("include-mixes", "value"),
    ]
)
def update_table_and_sunburst(rescue_type, include_mixes):
    if not rescue_type or rescue_type == "ALL":
        dff = df.copy()
    else:
        # start with dogs only
        dff = df[df["animal_type"] == "Dog"].copy()

        # age ≤ 2 years
        age_mask = dff["age_years"] <= 2

        # pure‐breed mask
        breeds    = BREED_MAP[rescue_type]
        pure_mask = dff["breed"].isin(breeds)

        # mix‐breed mask, if requested
        mix_mask = pd.Series(False, index=dff.index)
        if "MIXES" in include_mixes:
            for b in breeds:
                mix_mask |= (
                    dff["breed"].str.contains(b, case=False)
                    & dff["breed"].str.contains("mix", case=False)
                )

        # keep only dogs that satisfy age AND (pure OR mix)
        dff = dff[age_mask & (pure_mask | mix_mask)]

    # sunburst counts from the same filtered frame
    counts = (
        dff
        .groupby(["animal_type", "breed"])
        .size()
        .reset_index(name="count")
    )

    fig = px.sunburst(
        counts,
        path=["animal_type", "breed"],
        values="count",
        branchvalues="total",
        title="Animals by Type & Breed"
    )
    fig.update_layout(
        title_text="Animals by Type & Breed",
        title_x=0.5,
        title_font_color="white",
        paper_bgcolor="#2f2f2f",
        plot_bgcolor="#2f2f2f",
        margin=dict(t=40, l=0, r=0, b=0)
    )

    # return both the new table data and the updated figure
    return dff.to_dict("records"), fig

#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    # highlight selected columns
    selected = [{
        "if": {"column_id": col},
        "backgroundColor": "#D2F3FF"
    } for col in (selected_columns or [])]
    return selected


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    # geolocation chart
    dff = pd.DataFrame(viewData or [])
    row = index[0] if index else 0
    
    if dff.empty or row >= len(dff):
        return dl.Map(
            style={'width': '1000px', 'height': '500px'},
            # Default to center of Austin TX
            center=[30.75, -97.48],
            zoom=10,
            children=[dl.TileLayer(id="base-layer-id")]
        )

    # pull lat/lon and info from the selected row
    lat, lon = float(dff.iloc[row, 13]), float(dff.iloc[row, 14])
    breed = dff.iloc[row, 4]
    name  = dff.iloc[row, 9]
    
    # if there isn't a name, make the name "No Name"
    if len(name)<1:
        name = "No name"
    
    # From ModuleSix guidelines.
    return [
        dl.Map(style={'width': '1000px', 'height': '500px', 'margin': 'auto'},
           center=[lat,lon], zoom=10, children=[
           dl.TileLayer(id="base-layer-id"),
           dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
              children=[
                  dl.Tooltip(breed),
                  dl.Popup([
                     html.H1("Animal Name"),
                    html.H2(name)
             ])
          ])
       ])
    ]

app.run_server(debug=True)

Dash app running on http://127.0.0.1:12043/
